In [ ]:
%pip install ragatouille

In [ ]:
from glob import glob
lst_qa = glob("/kaggle/input/data-agent-qa*/*/*/*.json")
len(lst_qa)

In [2]:
from glob import glob
lst_txt = glob("/kaggle/input/data-agent-txt*/*/*/*.txt")
len(lst_txt)


1777

In [ ]:
import os
folder_dict = {}
for txt_path in lst_txt:
    folder_name = txt_path.split("/")[-2]
    if folder_name not in folder_dict:
        folder_dict[folder_name] = []
    folder_dict[folder_name].append(txt_path)

In [ ]:
document_dict = {}
for folder_name, lst_txt_path in folder_dict.items():
    lst_txt_path = sorted(lst_txt_path, key=lambda x: int(x[:-4].split("_")[-1]))
    full_text = ""
    for txt_path in lst_txt_path:
        with open(txt_path, "r") as fi:
            full_text += f"\n\n{fi.read()}"
    document_dict[folder_name] = full_text

In [ ]:
import json
from tqdm import tqdm
data_qa = []
for qa_path in tqdm(lst_qa):
    qa_data = json.load(open(qa_path, "r"))
    data_qa.extend([(obj['Question'].split("'")[0], obj['Answer']) for obj in qa_data])

print(len(data_qa))

In [ ]:
from ragatouille import RAGTrainer
from ragatouille.utils import get_wikipedia_page



# my_full_corpus = [get_wikipedia_page("Hayao_Miyazaki"), get_wikipedia_page("Studio_Ghibli")]


trainer = RAGTrainer(model_name = "MyFineTunedColBERT",
        pretrained_model_name = "colbert-ir/colbertv2.0", language_code="vi") # In this example, we run fine-tuning

# This step handles all the data processing, check the examples for more details!
# trainer.prepare_training_data(raw_data=data_qa,
#                                 )
trainer.prepare_training_data(
    raw_data = data_qa,
    data_out_path="/kaggle/working/data/",
    all_documents = list(document_dict.values()),
    num_new_negatives = 10,
    mine_hard_negatives= True,
    )

trainer.train(batch_size=32) # Train with the default hyperparams

In [ ]:
# !zip -r /kaggle/working/model.zip ragatouille/colbert/none/2025-02/22/10.47.56/checkpoints/colbert

In [ ]:
%ls /kaggle